In [2]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Align import MultipleSeqAlignment
from Bio.SeqRecord import SeqRecord
from Bio.Align.AlignInfo import SummaryInfo
import Bio
import os
import os.path
from collections import Counter
import pandas as pd
import re
from itertools import combinations
import numpy as np

In [2]:
def refine_id(string):
    components = string.split("_")
    seq_id = components[0]
    if seq_id[:2] == "IN" and seq_id[2].isdigit():
        seq_id = seq_id[:2] + "0" * (5 - len(seq_id)) + seq_id[2:]
    components[0] = seq_id
    return "_".join(components)

def dumb_consensus(sequences):
    consensus = Seq('')
    for col_num in range(len(sequences[0])):
        column = sequences[:, col_num]
        counts = Counter(column)
        most_frequent = counts.most_common(1)[0][0]
        consensus += most_frequent
        # print(f"position: {col_num}, nucleotides: {column}, consensus: {most_frequent}")
    return consensus

In [3]:
raw_tip_date_data = pd.read_excel("../Data/tip_dates/01-26-2016 Final list-upto IN729 update.xlsx")
tip_dates = raw_tip_date_data[["HIV Local ID", "Collection_Date", "HVR1-454"]]
tip_dates.columns = ["patient_id", "date", "type"]
tip_dates = tip_dates.dropna(subset=['date'])
tip_dates = tip_dates.fillna("")

In [4]:
seq_file = "../Data/from_alina/Indiana/cluster4_1a_threshold_5_ids_changed_mafft_nogaps.fas"
seq_name_pattern = re.compile(r"[A-Za-z]+\d+")

# m = re.search(seq_name_pattern, "3240IN499_")
# print(m.group(0))

sequences = list(SeqIO.parse(seq_file, 'fasta'))

type_mismatches = []
no_date = []
seq_df = None
selected_tip_dates = None
unique_patients = set()

for seq in sequences:
    patient_id = refine_id(re.search(seq_name_pattern, seq.id).group(0))
    if patient_id not in tip_dates["patient_id"].values:
        no_date.append(seq.id)
        continue

    tip_types = tip_dates.loc[tip_dates["patient_id"] == patient_id, "type"].iat[0].split("/")
    if tip_types != [''] and "1a" not in tip_types:
        type_mismatches.append(seq.id)
        continue

    tip_date = tip_dates.loc[tip_dates["patient_id"] == patient_id, "date"].iat[0]
    tip_record = pd.DataFrame([{"seq_id": seq.id, "date": tip_date}])
    if selected_tip_dates is None:
        selected_tip_dates = tip_record
    else:
        selected_tip_dates = pd.concat([selected_tip_dates, tip_record], ignore_index=True)
    unique_patients.add(patient_id)
    seq_record = pd.DataFrame([{"patient_id": patient_id, "sequence": seq}])
    if seq_df is None:
        seq_df = seq_record
    else:
        seq_df = pd.concat([seq_df, seq_record], ignore_index=True)


In [16]:
len(unique_patients)

46

In [37]:
output_folder = "../Data/from_alina/Indiana/"
tip_date_output_path = "/".join([output_folder, f"hcv_1a_tip_dates.txt"])

selected_tip_dates.to_csv(tip_date_output_path, sep="\t", index=False)

In [38]:

consensuses = []
consensus_tip_dates = None

for patient_id, group in seq_df.groupby("patient_id"):
    sequences = group["sequence"].to_list()
    alignment = MultipleSeqAlignment(sequences)
    consensus = dumb_consensus(alignment)
    consensus_formatted = SeqIO.SeqRecord(consensus, id=patient_id, description="")
    consensuses.append(consensus_formatted)

    tip_date = tip_dates.loc[tip_dates["patient_id"] == patient_id, "date"].iat[0]
    tip_record = pd.DataFrame([{"seq_id": patient_id, "date": tip_date}])
    if consensus_tip_dates is None:
        consensus_tip_dates = tip_record
    else:
        consensus_tip_dates = pd.concat([consensus_tip_dates, tip_record], ignore_index=True)

with open("../Data/consensus/hcv_1a.fas", 'w') as output_handle:
    SeqIO.write(consensuses, output_handle, 'fasta')
tip_date_output_path = "../Data/consensus/hcv_1a_tip_dates.txt"
consensus_tip_dates.to_csv(tip_date_output_path, sep="\t", index=False)
    

IN008
IN031
IN035
IN039
IN040
IN049
IN055
IN072
IN092
IN098
IN107
IN113
IN126
IN129
IN134
IN135
IN143
IN145
IN146
IN147
IN150
IN155
IN169
IN170
IN171
IN180
IN188
IN201
IN203
IN210
IN212
IN216
IN217
IN218
IN243
IN247
IN253
IN261
IN271
IN281
IN285
IN286
IN291
IN293
IN296
IN317
IN334
IN336
IN342
IN343
IN345
IN351
IN352
IN372
IN373
IN381
IN383
IN384
IN386
IN393
IN400
IN401
IN402
IN403
IN407
IN414
IN417
IN428
IN440
IN446
IN470
IN474
IN477
IN487
IN491
IN498
IN499
IN501
IN502
IN507
IN508
IN510
IN513
IN515


In [8]:
seq_file = "../Data/from_alina/Indiana/cluster4_3a_threshold_5_ids_changed_mafft_nogaps.fas"
seq_name_pattern = re.compile(r"[A-Za-z]+\d+")

# m = re.search(seq_name_pattern, "3240IN499_")
# print(m.group(0))

sequences = list(SeqIO.parse(seq_file, 'fasta'))

type_mismatches = []
no_date = []
seq_df = None
selected_tip_dates = None
unique_patients = set()

for seq in sequences:
    patient_id = refine_id(re.search(seq_name_pattern, seq.id).group(0))
    if patient_id not in tip_dates["patient_id"].values:
        no_date.append(seq.id)
        continue

    tip_types = tip_dates.loc[tip_dates["patient_id"] == patient_id, "type"].iat[0].split("/")
    if tip_types != [''] and "3a" not in tip_types:
        type_mismatches.append(seq.id)
        continue

    if not patient_id[2].isdigit():
        continue

    tip_date = tip_dates.loc[tip_dates["patient_id"] == patient_id, "date"].iat[0]
    tip_record = pd.DataFrame([{"seq_id": seq.id, "date": tip_date}])
    if selected_tip_dates is None:
        selected_tip_dates = tip_record
    else:
        selected_tip_dates = pd.concat([selected_tip_dates, tip_record], ignore_index=True)
    unique_patients.add(patient_id)
    seq_record = pd.DataFrame([{"patient_id": patient_id, "sequence": seq}])
    if seq_df is None:
        seq_df = seq_record
    else:
        seq_df = pd.concat([seq_df, seq_record], ignore_index=True)

In [40]:
output_folder = "../Data/from_alina/Indiana/"
tip_date_output_path = "/".join([output_folder, f"hcv_3a_tip_dates.txt"])

selected_tip_dates.to_csv(tip_date_output_path, sep="\t", index=False)

In [11]:

consensuses = []
consensus_tip_dates = None

for patient_id, group in seq_df.groupby("patient_id"):
    sequences = group["sequence"].to_list()
    alignment = MultipleSeqAlignment(sequences)
    consensus = dumb_consensus(alignment)
    consensus_formatted = SeqIO.SeqRecord(consensus, id=patient_id, description="")
    consensuses.append(consensus_formatted)

    tip_date = tip_dates.loc[tip_dates["patient_id"] == patient_id, "date"].iat[0]
    tip_record = pd.DataFrame([{"seq_id": patient_id, "date": tip_date}])
    if consensus_tip_dates is None:
        consensus_tip_dates = tip_record
    else:
        consensus_tip_dates = pd.concat([consensus_tip_dates, tip_record], ignore_index=True)

with open("../Data/consensus/hcv_3a.fas", 'w') as output_handle:
    SeqIO.write(consensuses, output_handle, 'fasta')
tip_date_output_path = "../Data/consensus/hcv_3a_tip_dates.txt"
consensus_tip_dates.to_csv(tip_date_output_path, sep="\t", index=False)
    

In [14]:
len(consensuses)

46

In [5]:
skyline_table_1a = pd.read_csv("../Beast/1a_skyline_data.txt", sep="\t")
skyline_table_3a = pd.read_csv("../Beast/3a_skyline_data.txt", sep="\t")

In [6]:
columns_to_sum = ["mean", "median", "upper", "lower"]
combined_df = skyline_table_1a.copy()
combined_df[columns_to_sum] = skyline_table_1a[columns_to_sum] + skyline_table_3a[columns_to_sum]


In [19]:
print(skyline_table_1a.head(20))
print(skyline_table_3a.head(20))
print(combined_df.head(20))

           time        date             datetime   milliseconds        mean  \
0   2015.800000  2015-10-19  2015-10-19T05:00:00  1445230800416   40.080446   
1   2015.765668  2015-10-06  2015-10-06T16:15:07  1444148107110   40.080446   
2   2015.731336  2015-09-24  2015-09-24T03:30:13  1443065413804   40.078660   
3   2015.697004  2015-09-11  2015-09-11T14:45:20  1441982720498   39.842178   
4   2015.662672  2015-08-30  2015-08-30T02:00:27  1440900027192   39.979945   
5   2015.628340  2015-08-17  2015-08-17T13:15:33  1439817333886   39.800771   
6   2015.594008  2015-08-05  2015-08-05T00:30:40  1438734640580   41.908105   
7   2015.559676  2015-07-23  2015-07-23T11:45:47  1437651947274   45.613235   
8   2015.525344  2015-07-10  2015-07-10T23:00:53  1436569253968   58.505959   
9   2015.491012  2015-06-28  2015-06-28T10:16:00  1435486560662   72.516648   
10  2015.456680  2015-06-15  2015-06-15T21:31:07  1434403867356   87.765338   
11  2015.422348  2015-06-03  2015-06-03T08:46:14  14

In [7]:
combined_df.to_csv("../Beast/combined_skyline_data.txt", sep="\t", index=False)